# Prueba Piloto

Librerías y Definiciones

In [1]:
#Librerías
import functions as fc

In [2]:
#Archivos
log_file = "logs.log"
connection_file = 'db_conection.json'
excel_file = 'prueba_carga_datos.xlsx'
map_file = 'columns_names_map.json'

In [3]:
#Deine Logs
fc.log_config(log_file)
#Establece conexión con BD
connection = fc.create_connection(connection_file)

06/14/2023 17:38:11 root            INFO     Log creado
06/14/2023 17:38:12 mysql.connector INFO     package: mysql.connector.plugins
06/14/2023 17:38:12 mysql.connector INFO     plugin_name: mysql_native_password
06/14/2023 17:38:12 mysql.connector INFO     AUTHENTICATION_PLUGIN_CLASS: MySQLNativePasswordAuthPlugin
06/14/2023 17:38:13 root            INFO     Conexión a Base de datos MySQL "piloto_v1" exitosa


Consulta tablas en BD

In [4]:
#SELECT
table_names = ['area', 'colaboradores_agencia']
for table in table_names:
    df_table = fc.select_all_table(table, connection)
    print(df_table.head(10),'\n')

06/14/2023 17:38:17 root            INFO     Consulta "SELECT * FROM area" exitosa


Empty DataFrame
Columns: [id_area, area, tipo, jefatura, objetivo, descripcion]
Index: [] 



06/14/2023 17:38:17 root            INFO     Consulta "SELECT * FROM colaboradores_agencia" exitosa


Empty DataFrame
Columns: [rut_colaborador, nombre, apellido_paterno, apellido_materno, cargo, id_area, genero, pueblo_originario, estado, f_ingreso, f_retiro]
Index: [] 



### Carga Datos Areas

Importación archivo de datos desde excel

In [6]:
sheet_name = 'area'
df_area = fc.import_data_excel(excel_file, sheet_name)
print('Primeras 15 filas del Dataframe cargado:')
df_area.head(15)

06/14/2023 17:38:35 root            INFO     Archivo: prueba_carga_datos.xlsx / hoja: "area" cargado correctamente


Primeras 15 filas del Dataframe cargado:


,ID,Tipo,Nombre,Sigla,Objetivo,Comentario,Jefatura
0,NaN,Área de Ejecución,A1,x,Objetivo,-,1.111.111-1
1,2.0,Área de Ejecución,A2,x,Objetivo,-,2.222.222-2
2,2.0,Área de Ejecución,A2 Duplicado,x,Objetivo,-,2.222.222-2
3,3.0,Área de Ejecución,A3,x,Objetivo,-,3.333.333-3
4,NaN,Oficina Temática,A4,x,Objetivo,-,4.444.444-4
5,5.0,Oficina Temática,A5,x,Objetivo,-,5.555.555-5
6,5.0,Oficina Temática,A5 Duplicado,x,Objetivo,-,5.555.555-5
7,6.0,Gerencia,A6,x,Objetivo,-,6.666.666-6
8,7.0,Gerencia,A7,x,Objetivo,-,7.777.777-7
9,7.0,Gerencia,A7 Duplicado,x,Objetivo,-,7.777.777-7


Tratamiento de datos

In [7]:
table_name = 'area'
df_area_clean = fc.clean_df(df_area, table_name, map_file, connection)
print('Primeras 10 filas del Dataframe:')
df_area_clean.head(10)

06/14/2023 17:38:49 root            INFO     Columnas renombradas: {'ID': 'id_area', 'Nombre': 'area', 'Tipo': 'tipo', 'Jefatura': 'jefatura', 'Objetivo': 'objetivo'}
06/14/2023 17:38:49 root            INFO     Filas nulas a descartar:
    id_area area               tipo     jefatura  objetivo
0      NaN   A1  Área de Ejecución  1.111.111-1  Objetivo
4      NaN   A4   Oficina Temática  4.444.444-4  Objetivo
06/14/2023 17:38:49 root            INFO     Filas duplicadas a descartar:
     id_area          area               tipo     jefatura  objetivo
1       2.0            A2  Área de Ejecución  2.222.222-2  Objetivo
5       5.0            A5   Oficina Temática  5.555.555-5  Objetivo
8       7.0            A7           Gerencia  7.777.777-7  Objetivo
9       7.0  A7 Duplicado           Gerencia  7.777.777-7  Objetivo
10      7.0  A7 Duplicado           Gerencia  7.777.777-7  Objetivo
06/14/2023 17:38:49 root            INFO     Nuevos tipos de datos:
   id_area    area    tipo jefatura 

Primeras 10 filas del Dataframe:


,id_area,area,tipo,jefatura,objetivo
1,2,A2,Área de Ejecución,2.222.222-2,Objetivo
3,3,A3,Área de Ejecución,3.333.333-3,Objetivo
5,5,A5,Oficina Temática,5.555.555-5,Objetivo
7,6,A6,Gerencia,6.666.666-6,Objetivo
8,7,A7,Gerencia,7.777.777-7,Objetivo


Construcción sentencia SQL + Insertar datos en BD

In [8]:
#Sentencia SQL
q1 = fc.insert_from_df(table_name, df_area_clean)
print(q1)
#Inserta datos
fc.execute_query(connection, q1)
df = fc.select_all_table (table_name, connection)
print('Primeras 10 filas en la BD:')
df.head(10)

06/14/2023 17:38:54 root            INFO     Generando consulta INSERT SQL



    INSERT INTO area (id_area, area, tipo, jefatura, objetivo)
    VALUES (2, 'A2', 'Área de Ejecución', '2.222.222-2', 'Objetivo'), (3, 'A3', 'Área de Ejecución', '3.333.333-3', 'Objetivo'), (5, 'A5', 'Oficina Temática', '5.555.555-5', 'Objetivo'), (6, 'A6', 'Gerencia', '6.666.666-6', 'Objetivo'), (7, 'A7', 'Gerencia', '7.777.777-7', 'Objetivo');
    


06/14/2023 17:38:55 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa
06/14/2023 17:38:55 root            INFO     Consulta "SELECT * FROM area" exitosa


Primeras 10 filas en la BD:


,id_area,area,tipo,jefatura,objetivo,descripcion
0,2,A2,Área de Ejecución,2.222.222-2,Objetivo,None
1,3,A3,Área de Ejecución,3.333.333-3,Objetivo,None
2,5,A5,Oficina Temática,5.555.555-5,Objetivo,None
3,6,A6,Gerencia,6.666.666-6,Objetivo,None
4,7,A7,Gerencia,7.777.777-7,Objetivo,None


Carga datos adicionales

In [9]:
sheet_name = 'area_2'
df_area_2 = fc.import_data_excel(excel_file, sheet_name)
df_area_clean_2 = fc.clean_df(df_area_2, table_name, map_file, connection)
print('Primeras 10 filas del Dataframe:')
df_area_clean_2.head(10)

06/14/2023 17:41:46 root            INFO     Archivo: prueba_carga_datos.xlsx / hoja: "area_2" cargado correctamente
06/14/2023 17:41:46 root            INFO     Columnas renombradas: {'ID': 'id_area', 'Nombre': 'area', 'Tipo': 'tipo', 'Jefatura': 'jefatura', 'Objetivo': 'objetivo'}
06/14/2023 17:41:46 root            INFO     Filas nulas a descartar:
    id_area area               tipo     jefatura  objetivo
0      NaN   A1  Área de Ejecución  1.111.111-1  Objetivo
4      NaN   A4   Oficina Temática  4.444.444-4  Objetivo
06/14/2023 17:41:46 root            INFO     Filas duplicadas a descartar:
     id_area          area               tipo     jefatura  objetivo
1       2.0            A2  Área de Ejecución  2.222.222-2  Objetivo
5       5.0            A5   Oficina Temática  5.555.555-5  Objetivo
8       7.0            A7           Gerencia  7.777.777-7  Objetivo
9       7.0  A7 Duplicado           Gerencia  7.777.777-7  Objetivo
10      7.0  A7 Duplicado           Gerencia  7.777.777

06/14/2023 17:41:46 root            INFO     Llaves Primarias existentes en BD a descartar:
 [2, 3, 5, 6, 7]


Primeras 10 filas del Dataframe:


,id_area,area,tipo,jefatura,objetivo
0,8,A8,Gerencia,8.888.888-8,Objetivo
1,9,A9,Gerencia,9.999.999-9,Objetivo


In [10]:
#Sentencia SQL
q2 = fc.insert_from_df(table_name, df_area_clean_2)
print(q2)
#Inserta datos
fc.execute_query(connection, q2)
df = fc.select_all_table (table_name, connection)
print('Primeras 10 filas en la BD:')
df.head(10)

06/14/2023 17:30:23 root            INFO     Generando consulta INSERT SQL



    INSERT INTO area (id_area, area, tipo, jefatura, objetivo)
    VALUES (8, 'A8', 'Gerencia', '8.888.888-8', 'Objetivo'), (9, 'A9', 'Gerencia', '9.999.999-9', 'Objetivo');
    


06/14/2023 17:30:24 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa
06/14/2023 17:30:24 root            INFO     Consulta "SELECT * FROM area" exitosa


Primeras 10 filas en la BD:


,id_area,area,tipo,jefatura,objetivo,descripcion
0,2,A2,Área de Ejecución,2.222.222-2,Objetivo,None
1,3,A3,Área de Ejecución,3.333.333-3,Objetivo,None
2,5,A5,Oficina Temática,5.555.555-5,Objetivo,None
3,6,A6,Gerencia,6.666.666-6,Objetivo,None
4,7,A7,Gerencia,7.777.777-7,Objetivo,None
5,8,A8,Gerencia,8.888.888-8,Objetivo,None
6,9,A9,Gerencia,9.999.999-9,Objetivo,None


### Carga Datos Colaboradores

Importación archivo de datos desde excel

In [10]:
excel_file = 'prueba_carga_datos.xlsx'
sheet_name = 'colaboradores_agencia'
table_name = 'colaboradores_agencia'

df_colab = fc.import_data_excel(excel_file, sheet_name)
print('Primeras 10 filas del Dataframe:')
df_colab.head(10)

06/14/2023 17:06:01 root            INFO     Archivo: prueba_carga_datos.xlsx / hoja: "colaboradores_agencia" cargado correctamente


Primeras 10 filas del Dataframe:


,Nombres,Apellido Paterno,Apellido Materno,Rut,Teléfono,Cargo,Area
0,Nombre 1,Apellido P1,NaN,1.111.111-1,1234567890,NaN,2
1,Nombre 2,NaN,Apellido M2,2.222.222-2,1234567890,NaN,2
2,NaN,Apellido P3,Apellido M3,3.333.333-3,1234567890,NaN,2
3,Nombre 4,Apellido P4,Apellido M4,4.444.444-4,1234567890,NaN,3
4,Nombre 5,Apellido P5,Apellido M5,5.555.555-5,1234567890,NaN,3
5,Nombre 6,Apellido P6,Apellido M6,NaN,1234567890,NaN,7
6,Nombre 7,Apellido P7,Apellido M7,7.777.777-7,1234567890,NaN,7
7,Nombre 7,Apellido P7,Apellido M7,7.777.777-7,1234567890,NaN,7


Tratamiento de datos

In [11]:
df_colab_clean = fc.clean_df(df_colab, table_name, map_file, connection)
df_colab_clean

06/14/2023 17:06:06 root            INFO     Columnas renombradas: {'Rut': 'rut_colaborador', 'Nombres': 'nombre', 'Apellido Paterno': 'apellido_paterno', 'Apellido Materno': 'apellido_materno', 'Cargo': 'cargo', 'Area': 'id_area'}
06/14/2023 17:06:06 root            INFO     Filas nulas a eliminar:
   rut_colaborador    nombre apellido_paterno apellido_materno  cargo  id_area
5             NaN  Nombre 6      Apellido P6      Apellido M6    NaN        7
06/14/2023 17:06:06 root            INFO     Filas duplicadas a eliminar:
   rut_colaborador    nombre apellido_paterno apellido_materno  cargo  id_area
6     7.777.777-7  Nombre 7      Apellido P7      Apellido M7    NaN        7
06/14/2023 17:06:06 root            INFO     Nuevos tipos de datos:
   rut_colaborador  nombre apellido_paterno apellido_materno   cargo id_area
0          object  object           object           object  object   int64


,rut_colaborador,nombre,apellido_paterno,apellido_materno,cargo,id_area
0,1.111.111-1,Nombre 1,Apellido P1,nan,nan,2
1,2.222.222-2,Nombre 2,nan,Apellido M2,nan,2
2,3.333.333-3,nan,Apellido P3,Apellido M3,nan,2
3,4.444.444-4,Nombre 4,Apellido P4,Apellido M4,nan,3
4,5.555.555-5,Nombre 5,Apellido P5,Apellido M5,nan,3
6,7.777.777-7,Nombre 7,Apellido P7,Apellido M7,nan,7


Construcción sentencia SQL + Inserta Datos

In [12]:
q2 = fc.insert_from_df(table_name, df_colab_clean)
print(q2)
fc.execute_query(connection, q2)
colab = fc.select_all_table (table_name, connection)
colab.head(10)

06/14/2023 17:07:00 root            INFO     Generando consulta INSERT SQL



    INSERT INTO colaboradores_agencia (rut_colaborador, nombre, apellido_paterno, apellido_materno, cargo, id_area)
    VALUES ('1.111.111-1', 'Nombre 1', 'Apellido P1', 'nan', 'nan', 2), ('2.222.222-2', 'Nombre 2', 'nan', 'Apellido M2', 'nan', 2), ('3.333.333-3', 'nan', 'Apellido P3', 'Apellido M3', 'nan', 2), ('4.444.444-4', 'Nombre 4', 'Apellido P4', 'Apellido M4', 'nan', 3), ('5.555.555-5', 'Nombre 5', 'Apellido P5', 'Apellido M5', 'nan', 3), ('7.777.777-7', 'Nombre 7', 'Apellido P7', 'Apellido M7', 'nan', 7);
    


06/14/2023 17:07:00 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa
06/14/2023 17:07:01 root            INFO     Consulta "SELECT * FROM colaboradores_agencia" exitosa


,rut_colaborador,nombre,apellido_paterno,apellido_materno,cargo,id_area,genero,pueblo_originario,estado,f_ingreso,f_retiro
0,1.111.111-1,Nombre 1,Apellido P1,nan,nan,2,None,None,None,None,None
1,2.222.222-2,Nombre 2,nan,Apellido M2,nan,2,None,None,None,None,None
2,3.333.333-3,nan,Apellido P3,Apellido M3,nan,2,None,None,None,None,None
3,4.444.444-4,Nombre 4,Apellido P4,Apellido M4,nan,3,None,None,None,None,None
4,5.555.555-5,Nombre 5,Apellido P5,Apellido M5,nan,3,None,None,None,None,None
5,7.777.777-7,Nombre 7,Apellido P7,Apellido M7,nan,7,None,None,None,None,None


Carga datos adicionales

In [13]:
sheet_name_2 = 'colaboradores_agencia_2'

df_colab_2 = fc.import_data_excel(excel_file, sheet_name_2)
print('Primeras 10 filas del Dataframe:')
df_colab_2.head(10)

06/14/2023 17:07:51 root            INFO     Archivo: prueba_carga_datos.xlsx / hoja: "colaboradores_agencia_2" cargado correctamente


Primeras 10 filas del Dataframe:


,Nombres,Apellido Paterno,Apellido Materno,Rut,Cargo,Area
0,Nombre 5,Apellido P5,Apellido M5,5.555.555-5,C1,5.0
1,Nombre 7,Apellido P7,Apellido M7,7.777.777-7,C2,5.0
2,Nombre 9,Apellido P9,Apellido M9,9.999.999-9,NaN,5.0
3,Nombre 11,Apellido P11,Apellido M11,1.222.333-4,NaN,5.0
4,Nombre 13,Apellido P13,Apellido M13,13.333.333-3,NaN,NaN


In [14]:
df_colab_2_clean = fc.clean_df(df_colab_2, table_name, map_file, connection)
df_colab_2_clean

06/14/2023 17:09:04 root            INFO     Columnas renombradas: {'Rut': 'rut_colaborador', 'Nombres': 'nombre', 'Apellido Paterno': 'apellido_paterno', 'Apellido Materno': 'apellido_materno', 'Cargo': 'cargo', 'Area': 'id_area'}
06/14/2023 17:09:04 root            INFO     Filas nulas a eliminar:
   rut_colaborador     nombre apellido_paterno apellido_materno cargo  id_area
4    13.333.333-3  Nombre 13     Apellido P13     Apellido M13   NaN      NaN
06/14/2023 17:09:04 root            INFO     Nuevos tipos de datos:
   rut_colaborador  nombre apellido_paterno apellido_materno   cargo id_area
0          object  object           object           object  object   int64
06/14/2023 17:09:05 root            INFO     Filas existentes en BD a eliminar:
                  nombre_x apellido_paterno_x apellido_materno_x cargo_x  \
rut_colaborador                                                           
5.555.555-5      Nombre 5        Apellido P5        Apellido M5     nan   
7.777.777-7    

,rut_colaborador,nombre,apellido_paterno,apellido_materno,cargo,id_area
0,9.999.999-9,Nombre 9,Apellido P9,Apellido M9,nan,5
1,1.222.333-4,Nombre 11,Apellido P11,Apellido M11,nan,5


In [15]:
q3 = fc.insert_from_df(table_name, df_colab_2_clean)
print(q3)
fc.execute_query(connection, q3)

06/14/2023 17:09:29 root            INFO     Generando consulta INSERT SQL



    INSERT INTO colaboradores_agencia (rut_colaborador, nombre, apellido_paterno, apellido_materno, cargo, id_area)
    VALUES ('9.999.999-9', 'Nombre 9', 'Apellido P9', 'Apellido M9', 'nan', 5), ('1.222.333-4', 'Nombre 11', 'Apellido P11', 'Apellido M11', 'nan', 5);
    


06/14/2023 17:09:29 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa


In [16]:
colab_2 = fc.select_all_table (table_name, connection)
colab_2.head(10)

06/14/2023 17:10:02 root            INFO     Consulta "SELECT * FROM colaboradores_agencia" exitosa


,rut_colaborador,nombre,apellido_paterno,apellido_materno,cargo,id_area,genero,pueblo_originario,estado,f_ingreso,f_retiro
0,1.111.111-1,Nombre 1,Apellido P1,nan,nan,2,None,None,None,None,None
1,1.222.333-4,Nombre 11,Apellido P11,Apellido M11,nan,5,None,None,None,None,None
2,2.222.222-2,Nombre 2,nan,Apellido M2,nan,2,None,None,None,None,None
3,3.333.333-3,nan,Apellido P3,Apellido M3,nan,2,None,None,None,None,None
4,4.444.444-4,Nombre 4,Apellido P4,Apellido M4,nan,3,None,None,None,None,None
5,5.555.555-5,Nombre 5,Apellido P5,Apellido M5,nan,3,None,None,None,None,None
6,7.777.777-7,Nombre 7,Apellido P7,Apellido M7,nan,7,None,None,None,None,None
7,9.999.999-9,Nombre 9,Apellido P9,Apellido M9,nan,5,None,None,None,None,None


In [11]:
#DELETE

table_name = "colaboradores_agencia"

q_delete = f""" 
DELETE FROM {table_name}
WHERE id_area  IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10);"""

fc.execute_query(connection, q_delete)
fc.select_all_table(table_name, connection)

06/14/2023 17:30:42 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa
06/14/2023 17:30:42 root            INFO     Consulta "SELECT * FROM colaboradores_agencia" exitosa


,rut_colaborador,nombre,apellido_paterno,apellido_materno,cargo,id_area,genero,pueblo_originario,estado,f_ingreso,f_retiro


In [12]:
#DELETE

table_name = "area"

q_delete = f""" 
DELETE FROM {table_name}
WHERE id_area  IN (1, 2, 3, 4, 5, 6, 7, 8, 9, 10);"""

fc.execute_query(connection, q_delete)
fc.select_all_table(table_name, connection)

06/14/2023 17:30:45 root            INFO     Consulta INSERT/UPDATE/DELETE exitosa
06/14/2023 17:30:45 root            INFO     Consulta "SELECT * FROM area" exitosa


,id_area,area,tipo,jefatura,objetivo,descripcion
